In [1]:
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm


#import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from IPython.display import clear_output 
import time
import os
import json
import random

random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [2]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    if mask_rle == -1:
        return np.zeros(shape, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    if np.sum(mask) == 0:
        return '-1'
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [3]:
csv_files = []
csv_files.append(pd.read_csv('../submit/b5_jhsstride_040_yjypost.csv'))  # CSV 파일 경로
csv_files.append(pd.read_csv('../submit/submit_deepsplit_post.csv'))
#csv_files.append(pd.read_csv('../submit/b1_512size_jhsstride_040.csv'))

# 결과를 저장할 새로운 DataFrame 생성
result_data = pd.DataFrame(columns=['img_id', 'mask_rle'])

hflencsv=len(csv_files) // 2

for i in tqdm(range(60640)):
    sumask=np.zeros((224,224), dtype=np.uint8)
    for csv_file in csv_files:
        mask_data = csv_file['mask_rle'][i]
        mask = rle_decode(mask_data, (224,224))
        sumask += mask
    sumask=np.where(sumask > hflencsv,1,0)

    result_data.loc[i] = [csv_file['img_id'][i], rle_encode(sumask)]
print(result_data.head)


100%|██████████| 60640/60640 [05:59<00:00, 168.85it/s]

<bound method NDFrame.head of            img_id                                           mask_rle
0      TEST_00000  20002 8 20226 9 20450 9 20481 6 20673 11 20704...
1      TEST_00001  36060 5 36282 7 36505 8 36727 10 36950 11 3717...
2      TEST_00002                                                 -1
3      TEST_00003  22 23 73 12 245 25 297 12 467 27 521 10 690 29...
4      TEST_00004  16891 20 17114 24 17148 11 17338 46 17562 46 1...
...           ...                                                ...
60635  TEST_60635                                                 -1
60636  TEST_60636  39420 1 39636 9 39856 14 40078 17 40301 19 405...
60637  TEST_60637  88 18 312 18 536 18 760 18 984 18 1208 17 1432...
60638  TEST_60638  14706 11 14928 14 15148 18 15371 20 15594 21 1...
60639  TEST_60639  11892 9 12115 9 12337 11 12559 13 12782 14 130...

[60640 rows x 2 columns]>


In [7]:
result_data.to_csv('../submit/ensemble/ensemble1.csv', index=False)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
